In [5]:
# %matplotlib
# %matplotlib inline
# %matplotlib notebook

import pandas as pd
import numpy as np
import random
import os
import warnings
from datetime import datetime, timedelta, timezone
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FormatStrFormatter
import re
import math
from copy import deepcopy
from numba_stats import t
import scipy.stats as stats

# IMM
from gridmeter.individual_meter_matching.create_comparison_groups import (
    Individual_Meter_Matching as IMM,
)
from gridmeter.individual_meter_matching import settings as IMM_settings

# Clustering
from gridmeter.clustering import (
    settings as clustering_settings,
    cluster as clustering,
    transform as clustering_transform,
)

from IPython.display import Image, Markdown, display
plt.ion()
plt.rcParams['figure.figsize'] = [24, 16]
plt.rcParams['figure.dpi'] = 300

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df_ls_t = pd.read_csv("/app/.recurve_cache/clustering/example_dfs/df_ls_t.csv")
df_ls_cp = pd.read_csv("/app/.recurve_cache/clustering/example_dfs/df_ls_cp.csv")

In [8]:
# Test IMM

df_ls_cp_mod = df_ls_cp.set_index(["id", "hour"]).unstack()
df_ls_cp_mod.columns = df_ls_cp_mod.columns.droplevel(0)

df_ls_t_mod = df_ls_t.set_index(["id", "hour"]).unstack()
df_ls_t_mod.columns = df_ls_t_mod.columns.droplevel(0)

imm_settings = IMM_settings.Settings()
df_cg, df_t_coeffs = IMM(imm_settings).get_comparison_group(df_ls_t_mod, df_ls_cp_mod)
df_cg

,treatment,distance,duplicated,cluster
id,,,,
None-3234991905-3234991905,None-1094275585-1094275585,1.488844,False,1
None-1518448307-1518448307,None-1397301805-1397301805,4.454165,False,1
None-3517431310-3517431310,None-1432022910-1432022910,4.414845,False,1
None-1908503005-1908503005,None-1469355610-1469355610,1.487711,False,1
None-3529331605-3529331605,None-1504812305-1504812305,0.969864,False,1
...,...,...,...,...
None-1802814010-1802814010,None-5519977972-5519977972,80.664935,False,1
None-1646832205-1646832205,None-8098217928-8098217928,18.023014,False,1
None-0603795751-0603795751,None-8313277985-8313277985,9.958277,False,1


In [35]:
df_ls_cp_mod

hour,1,2,3,4,5,6,7,8,9,10,...,495,496,497,498,499,500,501,502,503,504
id,,,,,,,,,,,,,,,,,,,,,
None-0344817417-0344817417,0.048667,0.038093,0.029257,0.038400,0.031701,0.017867,-0.006818,-0.017955,-0.016814,0.002400,...,0.000917,-0.001362,0.030633,0.040428,0.046844,0.016800,0.008000,-0.012496,-0.029353,-0.011955
None-0356376858-0356376858,0.003630,-0.005249,0.002243,-0.009134,-0.003112,-0.008456,-0.007128,-0.000260,0.010316,-0.005634,...,0.054740,0.069601,0.054557,0.038718,0.025816,0.072563,0.052087,-0.003991,0.020637,0.035314
None-0490610165-0490610165,0.002965,0.044746,-0.003831,0.000396,0.025132,-0.019402,-0.027871,0.011252,-0.027059,-0.031402,...,0.047398,0.057855,0.035343,0.062176,0.032220,-0.010008,-0.006652,0.086482,0.078283,0.114412
None-0497060249-0497060249,0.004046,0.041287,0.048357,0.032730,0.033340,0.028570,0.064851,0.054879,0.044811,-0.004200,...,0.084856,0.073313,0.101408,0.072407,0.055214,-0.005878,-0.066201,-0.025051,-0.019424,-0.034227
None-0567408984-0567408984,0.020286,0.001132,-0.008245,0.000370,0.015974,0.009853,-0.013497,-0.014468,-0.008707,-0.005385,...,0.030539,-0.002812,-0.028070,0.012364,0.052373,0.018119,0.025093,0.058662,0.098191,0.104237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None-9467861419-9467861419,0.942396,0.307870,0.293977,0.280960,0.287584,0.298102,0.003597,0.106951,0.121297,-0.010081,...,-0.077094,-0.179914,-0.190653,0.009298,0.009534,-0.051591,-0.018354,0.046122,-0.355331,0.318566
None-9552364372-9552364372,0.026223,-0.008756,-0.002434,-0.017950,-0.032235,-0.034844,-0.054470,-0.052005,-0.008880,0.016404,...,0.071608,0.097168,0.054964,0.047771,0.022744,0.033969,0.028445,0.009381,0.049313,0.007954
None-9576055824-9576055824,-0.070576,-0.024393,0.002843,-0.023571,-0.040715,0.028543,0.069476,-0.016303,-0.147807,-0.225966,...,-0.028787,0.006562,0.044314,0.065441,0.048302,0.035267,0.043528,0.049412,0.067204,0.062055


In [34]:
from gridmeter.utils.calculate_distances import calculate_distances

def TestDistanceMatching(
    df_ls_t,
    df_ls_c,
    n_matches_per_treatment=4,
    distance_metric="euclidean",
    allow_duplicate_match=False,
    replace_duplicate_method=None,  # currently unused [None, "closest_to_meter", "closest_global"]
    max_distance_threshold=None,
    n_match_multiplier=None,
    n_meters_per_chunk=10000,
):
    ls_t = df_ls_t.to_numpy()
    ls_cp = df_ls_c.to_numpy()

    # Calculate closest distances
    if n_match_multiplier is None:
        n_matches_per_treatment = None

    if n_match_multiplier is not None:
        if (not allow_duplicate_match and replace_duplicate_method is not None) or max_distance_threshold is not None:
            n_matches_per_treatment *= n_match_multiplier

        if n_matches_per_treatment > ls_cp.shape[0]:
            n_matches_per_treatment = ls_cp.shape[0]

    cp_id_idx, dist = calculate_distances(
        ls_t, ls_cp, distance_metric, n_matches_per_treatment, n_meters_per_chunk
    )

    # create dataframes
    id_t = df_ls_t.index.values
    id_c = df_ls_c.index.values

    series_t = pd.Series(np.repeat(id_t, dist.shape[1]), name="treatment")
    series_cp = pd.Series(id_c[cp_id_idx.flatten()], name="id")
    clusters = pd.DataFrame(
        dist.flatten(), index=[series_t, series_cp], columns=["distance"]
    )
    clusters = clusters.reset_index()
    clusters["duplicated"] = clusters.duplicated(subset=["id"])
    clusters["cluster"] = 1
    clusters = clusters.set_index("id")

    if not allow_duplicate_match:
        # drop duplicate index
        clusters = clusters[~clusters.index.duplicated(keep='first')]

        # get count of treatment
        print(clusters['treatment'])
        counts = clusters['treatment'].value_counts()
        print(counts)

        if replace_duplicate_method is not None:
            raise NotImplementedError(
                "'replace_duplicate_meters': True not implemented"
            )

    return clusters


def get_comparison_group(df_ls_t, df_ls_cp, weights=None):
    df_cg = TestDistanceMatching(df_ls_t, df_ls_cp)

    # Create df_t_coeffs
    t_ids = df_ls_t.index.unique()
    coeffs = np.ones(t_ids.values.size)

    df_t_coeffs = pd.DataFrame(coeffs, index=t_ids, columns=["pct_cluster_1"])
    df_t_coeffs.index.name = "id"

    return df_cg, df_t_coeffs

In [33]:
df_cg, df_t_coeffs = get_comparison_group(df_ls_t_mod, df_ls_cp_mod)
df_cg

id
None-0344817417-0344817417    None-1094275585-1094275585
None-0356376858-0356376858    None-1094275585-1094275585
None-0490610165-0490610165    None-1094275585-1094275585
None-0497060249-0497060249    None-1094275585-1094275585
None-0567408984-0567408984    None-1094275585-1094275585
                                         ...            
None-9467861419-9467861419    None-1094275585-1094275585
None-9552364372-9552364372    None-1094275585-1094275585
None-9576055824-9576055824    None-1094275585-1094275585
None-9808840282-9808840282    None-1094275585-1094275585
None-9963816099-9963816099    None-1094275585-1094275585
Name: treatment, Length: 1000, dtype: object
treatment
None-1094275585-1094275585    1000
Name: count, dtype: int64


,treatment,distance,duplicated,cluster
id,,,,
None-0344817417-0344817417,None-1094275585-1094275585,1.689297,False,1
None-0356376858-0356376858,None-1094275585-1094275585,1.710408,False,1
None-0490610165-0490610165,None-1094275585-1094275585,7.299609,False,1
None-0497060249-0497060249,None-1094275585-1094275585,19.960519,False,1
None-0567408984-0567408984,None-1094275585-1094275585,1.583618,False,1
...,...,...,...,...
None-9467861419-9467861419,None-1094275585-1094275585,1354.884431,False,1
None-9552364372-9552364372,None-1094275585-1094275585,1.770874,False,1
None-9576055824-9576055824,None-1094275585-1094275585,7.978226,False,1


In [12]:
df_cg.sort_values(["treatment", "distance"])

,treatment,distance,duplicated,cluster
id,,,,
None-3234991905-3234991905,None-1094275585-1094275585,1.488844,False,1
None-4336129705-4336129705,None-1094275585-1094275585,1.518834,False,1
None-3735344605-3735344605,None-1094275585-1094275585,1.539653,False,1
None-3599514705-3599514705,None-1094275585-1094275585,1.540608,False,1
None-3771957305-3771957305,None-1094275585-1094275585,1.546059,False,1
...,...,...,...,...
None-1820004105-1820004105,None-1094275585-1094275585,695.012382,False,1
None-3103067310-3103067310,None-1094275585-1094275585,916.604475,False,1
None-9467861419-9467861419,None-1094275585-1094275585,1354.884431,False,1


In [37]:
# Test Clustering

clustering_settings = clustering_settings.Settings()
matcher = clustering.ClusterResult.from_comparison_pool_loadshapes_and_settings(
    df_cp_ls=df_ls_cp, s=clustering_settings
)
treatment_df = matcher.get_match_treatment_to_cluster_df(
    treatment_loadshape_df=df_ls_t
)

print(matcher.cluster_df)
print(treatment_df)

AttributeError: 'Settings' object has no attribute 'Settings'

In [31]:
from utils.data_settings import Settings

class Data:
    def __init__(self, settings : Settings | None):
        if settings is None:
            settings = Settings()
        else:
            self.settings = {"agg_type": "mean",
                            "loadshape_type": "observed", # ["observed", "modeled", "error"]
                            "time_period": "hour", #["hour", "week", "season"]
                            }

    def convert_timeseries_to_loadshape(self, df):
        # Check columns missing in time_series_df
        expected_columns = ["id", "datetime", self.settings["loadshape_type"]] # except error which requires both observed and modeled
        missing_columns = [c for c in expected_columns if c not in time_series_df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing columns in time_series_df: {missing_columns}")
        
        # df_type
        if self.settings["loadshape_type"] == "observed":
            df_type = "observed"
        elif self.settings["loadshape_type"] == "modeled":
            df_type = "modeled"
        elif self.settings["loadshape_type"] == "error":
            df_type = "error"
        else:
            raise ValueError(f"Invalid loadshape_type: {self.settings['loadshape_type']}")
            

        if df_type == "error":
            pass # calculate error

        # Aggregate the input time_series based on time_period
        time_series_df = time_series_df.set_index("datetime")
        time_series_df = time_series_df.groupby(["id", pd.Grouper(freq=self.settings["time_period"])])[df_type].agg(self.settings["agg_type"])
        time_series_df = time_series_df.reset_index()
        time_series_df = time_series_df.rename(columns={"datetime": "time"})
        time_series_df["loadshape"] = time_series_df[df_type]
        time_series_df = time_series_df.drop(columns=[df_type])
        time_series_df = time_series_df.set_index(["id", "time"])
        loadshape_df = time_series_df
        
        return loadshape_df
    
    def _validate(self, df):
        pass

    
    def set_data(self, loadshape_df=None, time_series_df=None):
        """

        Args:
            Loadshape_df: columns = [id, time, loadshape]

            Time_series_df: columns = [id, datetime, observed, observed_error, modeled, modeled_error]

        Output:
            loadshape: idx = id, columns = time, columns = [loadshape]

            
        """
        if loadshape_df is None and time_series_df is None:
            raise ValueError("Either loadshape_dataframe or time_series_dataframe must be provided.")


        if loadshape_df is not None:
            # Check columns missing in loadshape_df
            expected_columns = ["id", "time", "loadshape"]
            missing_columns = [c for c in expected_columns if c not in loadshape_df.columns]
            
            if missing_columns:
                raise ValueError(f"Missing columns in time_series_df: {missing_columns}")

            
            #Aggregate the input loadshape based on time_period
            output_loadshape = loadshape_df.groupby(["id", pd.Grouper(freq=self.settings["time_period"])])["loadshape"].agg(self.settings["agg_type"])
            
        elif time_series_df is not None:
            output_loadshape = convert_timeseries_to_loadshape(time_series_df)

        
        self.loadshape = output_loadshape


        return self.loadshape

        

In [36]:
df_ls_t_mod.head()

hour,1,2,3,4,5,6,7,8,9,10,...,495,496,497,498,499,500,501,502,503,504
id,,,,,,,,,,,,,,,,,,,,,
None-1094275585-1094275585,-0.004892,0.014424,0.024523,0.009783,0.002749,-0.018928,0.000177,-0.022380,-0.015270,-0.015309,...,-0.072218,-0.048825,-0.046101,-0.036118,-0.021544,0.041530,0.057548,0.083471,0.063300,0.073510
None-1397301805-1397301805,-0.040984,-0.017669,-0.000320,0.023877,0.026313,-0.035445,0.017402,0.097946,0.067614,0.139950,...,0.065461,0.161034,0.069464,-0.014588,0.022082,0.088210,-0.039004,0.088822,-0.021783,0.020953
None-1432022910-1432022910,0.005879,0.007890,-0.005875,0.002565,0.007195,-0.019074,-0.007493,-0.007334,-0.018340,-0.040356,...,0.103594,0.077896,0.055153,0.013053,-0.002923,0.015454,0.037913,0.038603,0.019493,0.046673
None-1469355610-1469355610,-0.005751,-0.002495,0.000473,0.001568,-0.008138,-0.027553,-0.007281,0.051084,-0.008178,0.020144,...,0.001396,-0.059681,0.053881,0.055815,0.067706,0.049554,0.029438,0.031970,0.144222,0.123939
None-1504812305-1504812305,0.042894,0.010071,-0.028050,-0.014719,-0.021681,-0.016993,-0.045222,-0.004814,-0.008289,-0.002940,...,0.161389,0.164025,0.106125,0.026877,0.010182,0.031870,-0.010856,0.013376,0.037637,0.020543


In [32]:
data = Data(settings = None)
print(data.set_data(loadshape_df=df_ls_t))

ValueError: Missing columns in time_series_df: ['time', 'loadshape']